# Prepare dataset

- Create dataset from the new dataframe

## Librairies

In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import os
import pickle

import pandas as pd

import numpy as np
import torch

import transformers
from transformers import BertTokenizer

import datasets
from datasets import concatenate_datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import DatasetDict

In [3]:
print('pandas:\t\t', pd.__version__)
print('transformers:\t', transformers.__version__)
print('datasets:\t', datasets.__version__)

pandas:		 1.3.4
transformers:	 4.12.5
datasets:	 1.15.1


## Global variables

In [4]:
dataset_df = pd.read_pickle("/notebooks/token_classification/notebooks/TokCL_w_POS/pe_w_para_pos_joint_tags_pickle")

In [5]:
dataset_df

,split,essay,essay_nr,para_nr,paragraph,paragraph_words,paragraph_tags,tags_list_int,paragraph_tags_bio_only,paragraph_tags_bio_only_int,paragraph_tags_bio_only_correction,paragraph_pos_tags,is_equal_tag,joint_tags
0,TRAIN,Should students be taught to compete or to coo...,essay001,1,It is always said that competition can effecti...,"[It, is, always, said, that, competition, can,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[PRON, AUX, ADV, VERB, DET, NOUN, VERB, ADV, V...",1,"[It, PRON, is, AUX, always, ADV, said, VERB, t..."
1,TRAIN,Should students be taught to compete or to coo...,essay001,2,"First of all, through cooperation, children ca...","[First, of, all, ,, through, cooperation, ,, c...","[O, O, O, O, B-Claim, I-Claim, I-Claim, I-Clai...","[0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[O, O, O, O, B, I, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, O, B-COMP, I-COMP, I-COMP, I-COMP, I...","[ADV, ADP, DET, PUNCT, ADP, NOUN, PUNCT, NOUN,...",1,"[First, ADV, of, ADP, all, DET, ,, PUNCT, thro..."
2,TRAIN,Should students be taught to compete or to coo...,essay001,3,"On the other hand, the significance of competi...","[On, the, other, hand, ,, the, significance, o...","[O, O, O, O, O, B-Premise, I-Premise, I-Premis...","[0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[O, O, O, O, O, B, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, O, O, B-COMP, I-COMP, I-COMP, I-COMP...","[ADP, DET, ADJ, NOUN, PUNCT, DET, NOUN, ADP, N...",1,"[On, ADP, the, DET, other, ADJ, hand, NOUN, ,,..."
3,TRAIN,Should students be taught to compete or to coo...,essay001,4,"Consequently, no matter from the view of indiv...","[Consequently, ,, no, matter, from, the, view,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[ADV, PUNCT, ADV, ADV, ADP, DET, NOUN, ADP, AD...",1,"[Consequently, ADV, ,, PUNCT, no, ADV, matter,..."
4,TRAIN,More people are migrating to other countries t...,essay002,1,The last 50 years have seen an increasing numb...,"[The, last, 50, years, have, seen, an, increas...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[DET, ADJ, NUM, NOUN, AUX, VERB, DET, VERB, NO...",1,"[The, DET, last, ADJ, 50, NUM, years, NOUN, ha..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,TRAIN,Fatherhood should be as present as motherhood ...,essay401,3,The second reason why I believe that fatherood...,"[The, second, reason, why, I, believe, that, f...","[B-Claim, I-Claim, I-Claim, I-Claim, I-Claim, ...","[3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[B-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[DET, ADJ, NOUN, ADV, PRON, VERB, SCONJ, NOUN,...",1,"[The, DET, second, ADJ, reason, NOUN, why, ADV..."
1368,TRAIN,Fatherhood should be as present as motherhood ...,essay401,4,"In conclusion, fatherhood should be as present...","[In, conclusion, ,, fatherhood, should, be, as...","[O, O, O, B-MajorClaim, I-MajorClaim, I-MajorC...","[0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, B, I, I, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, B-COMP, I-COMP, I-COMP, I-COMP, I-CO...","[ADP, NOUN, PUNCT, ADJ, VERB, AUX, ADV, ADJ, S...",1,"[In

In [7]:
dataset_df['joint_tags'][0]

['It, PRON',
 'is, AUX',
 'always, ADV',
 'said, VERB',
 'that, DET',
 'competition, NOUN',
 'can, VERB',
 'effectively, ADV',
 'promote, VERB',
 'the, DET',
 'development, NOUN',
 'of, ADP',
 'economy, NOUN',
 '., PUNCT',
 'In, ADP',
 'order, NOUN',
 'to, PART',
 'survive, VERB',
 'in, ADP',
 'the, DET',
 'competition, NOUN',
 ',, PUNCT',
 'companies, NOUN',
 'continue, VERB',
 'to, PART',
 'improve, VERB',
 'their, DET',
 'products, NOUN',
 'and, CCONJ',
 'service, NOUN',
 ',, PUNCT',
 'and, CCONJ',
 'as, SCONJ',
 'a, DET',
 'result, NOUN',
 ',, PUNCT',
 'the, DET',
 'whole, ADJ',
 'society, NOUN',
 'prospers, NOUN',
 '., PUNCT',
 'However, ADV',
 ',, PUNCT',
 'when, ADV',
 'we, PRON',
 'discuss, VERB',
 'the, DET',
 'issue, NOUN',
 'of, ADP',
 'competition, NOUN',
 'or, CCONJ',
 'cooperation, NOUN',
 ',, PUNCT',
 'what, PRON',
 'we, PRON',
 'are, AUX',
 'concerned, ADJ',
 'about, ADP',
 'is, AUX',
 'not, PART',
 'the, DET',
 'whole, ADJ',
 'society, NOUN',
 ',, PUNCT',
 'but, CCONJ'

In [8]:
dataset_df.columns

Index(['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words',
       'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only',
       'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction',
       'paragraph_pos_tags', 'is_equal_tag', 'joint_tags'],
      dtype='object')

In [9]:
# sanity check
print(len(dataset_df))
dataset_df = dataset_df.dropna()
print(len(dataset_df))

1372
1372


In [10]:
dataset_df = dataset_df.reset_index(drop=True)

In [11]:
dataset_df

,split,essay,essay_nr,para_nr,paragraph,paragraph_words,paragraph_tags,tags_list_int,paragraph_tags_bio_only,paragraph_tags_bio_only_int,paragraph_tags_bio_only_correction,paragraph_pos_tags,is_equal_tag,joint_tags
0,TRAIN,Should students be taught to compete or to coo...,essay001,1,It is always said that competition can effecti...,"[It, is, always, said, that, competition, can,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[PRON, AUX, ADV, VERB, DET, NOUN, VERB, ADV, V...",1,"[It, PRON, is, AUX, always, ADV, said, VERB, t..."
1,TRAIN,Should students be taught to compete or to coo...,essay001,2,"First of all, through cooperation, children ca...","[First, of, all, ,, through, cooperation, ,, c...","[O, O, O, O, B-Claim, I-Claim, I-Claim, I-Clai...","[0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[O, O, O, O, B, I, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, O, B-COMP, I-COMP, I-COMP, I-COMP, I...","[ADV, ADP, DET, PUNCT, ADP, NOUN, PUNCT, NOUN,...",1,"[First, ADV, of, ADP, all, DET, ,, PUNCT, thro..."
2,TRAIN,Should students be taught to compete or to coo...,essay001,3,"On the other hand, the significance of competi...","[On, the, other, hand, ,, the, significance, o...","[O, O, O, O, O, B-Premise, I-Premise, I-Premis...","[0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[O, O, O, O, O, B, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, O, O, B-COMP, I-COMP, I-COMP, I-COMP...","[ADP, DET, ADJ, NOUN, PUNCT, DET, NOUN, ADP, N...",1,"[On, ADP, the, DET, other, ADJ, hand, NOUN, ,,..."
3,TRAIN,Should students be taught to compete or to coo...,essay001,4,"Consequently, no matter from the view of indiv...","[Consequently, ,, no, matter, from, the, view,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[ADV, PUNCT, ADV, ADV, ADP, DET, NOUN, ADP, AD...",1,"[Consequently, ADV, ,, PUNCT, no, ADV, matter,..."
4,TRAIN,More people are migrating to other countries t...,essay002,1,The last 50 years have seen an increasing numb...,"[The, last, 50, years, have, seen, an, increas...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[DET, ADJ, NUM, NOUN, AUX, VERB, DET, VERB, NO...",1,"[The, DET, last, ADJ, 50, NUM, years, NOUN, ha..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,TRAIN,Fatherhood should be as present as motherhood ...,essay401,3,The second reason why I believe that fatherood...,"[The, second, reason, why, I, believe, that, f...","[B-Claim, I-Claim, I-Claim, I-Claim, I-Claim, ...","[3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[B-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[DET, ADJ, NOUN, ADV, PRON, VERB, SCONJ, NOUN,...",1,"[The, DET, second, ADJ, reason, NOUN, why, ADV..."
1368,TRAIN,Fatherhood should be as present as motherhood ...,essay401,4,"In conclusion, fatherhood should be as present...","[In, conclusion, ,, fatherhood, should, be, as...","[O, O, O, B-MajorClaim, I-MajorClaim, I-MajorC...","[0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, B, I, I, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, B-COMP, I-COMP, I-COMP, I-COMP, I-CO...","[ADP, NOUN, PUNCT, ADJ, VERB, AUX, ADV, ADJ, S...",1,"[In

In [12]:
train_df = dataset_df[dataset_df.split=='TRAIN'].reset_index(drop=True)
test_df = dataset_df[dataset_df.split=='TEST'].reset_index(drop=True)



# for example

# val_df = dataset_df[dataset_df.split=='VALIDATION'].reset_index(drop=True)

## Prepare Dataset object

In [13]:
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

In [14]:
train_val_datasets = dataset_train.train_test_split(train_size=0.8)
dataset_train = train_val_datasets['train']
dataset_val = train_val_datasets['test']

In [15]:
dataset = DatasetDict({"train": dataset_train, "test": dataset_test, "validation": dataset_val})

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction', 'paragraph_pos_tags', 'is_equal_tag', 'joint_tags'],
        num_rows: 868
    })
    test: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction', 'paragraph_pos_tags', 'is_equal_tag', 'joint_tags'],
        num_rows: 286
    })
    validation: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction', 'paragraph_pos_tags', 'is_equal_tag', 'joint_tags'],
        num_rows: 218
    })
})

In [17]:
# sanity check
set(dataset['train']['split']), set(dataset['test']['split']), set(dataset['validation']['split'])

({'TRAIN'}, {'TEST'}, {'TRAIN'})

## Save dataset

In [18]:
torch.save(dataset, os.path.join("/notebooks/token_classification/notebooks/TokCL_w_POS", 'pe_w_para_pos_joint_tags.pt'))

In [17]:
with open(os.path.join("/notebooks/token_classification/data/", 'pe_dataset_task_1_bio_only.pt'), "wb") as f:
    pickle.dump(dataset, f)
#torch.save(dataset, os.path.join("/notebooks/token_classification/data/", 'pe_dataset_task_1_bio_only.pt'))